# Читаем обучающий датасет и делаем стратифицированное разбиение


dev датасет составляет примерно 5% от трейна. 

Для валидации отпилю 15% выборки, должно хватить. 

In [1]:
import numpy as np
import random

In [2]:
random.seed(17)
np.random.seed(17)

In [3]:
import json

with open('../data/train_dataset.json') as f:
    data = json.load(f)

Копипаста с ноутбука с EDA

In [4]:
from collections import Counter

import pandas as pd


def count_entities(sample):
    collapsed_entities = []
    saw_begin_before = False

    for label in sample['token_labels']:
        if label == 'O':
            saw_begin_before = False
        elif label[0] == 'B':
            saw_begin_before = True
            collapsed_entities.append(label[2:])
        elif label[0] == 'I':
            if saw_begin_before:
                continue
            else:
                raise ValueError("Found I-label without B-label before")
    
    c = dict(Counter(collapsed_entities))
    c['lang'] = sample['lang']
    return c


entity_counts = pd.DataFrame(
    [count_entities(sample) for sample in data]
).fillna(0)

Бить будем по текстам, а не по сущностям. Поэтому нам нужно привести все счетчики сущностей к бинарному виду

In [5]:
entity_names = ['GRP', 'PER', 'CW', 'PROD', 'CORP', 'LOC']

entity_counts[entity_names] = (entity_counts[entity_names] > 0).astype(int)

In [6]:
entity_counts.head()

,CORP,lang,GRP,PER,CW,PROD,LOC
0,1,BN-Bangla,0,0,0,0,0
1,0,BN-Bangla,1,0,0,0,0
2,0,BN-Bangla,0,1,0,0,0
3,1,BN-Bangla,0,0,0,0,0
4,0,BN-Bangla,1,0,0,0,0


Стратицифированное разбиение из sklearn работает с одним столбцом "целевая переменная". Нам это не подходит, сделаю свою реализацию. 


Идеально провести разбиение, судя по всему, не получится, поэтому сделаем "перекос" максимально близким к тому, что наблюдается в train/dev

In [7]:
langmeans_diffs = pd.read_csv('../data/processed_data/langmeans_diffs.csv')

In [8]:
langmeans_diffs.head()

,lang,CORP,GRP,PER,CW,PROD,LOC
0,BN-Bangla,0.010989,0.009690,0.009673,0.009020,0.029134,0.027149
1,DE-German,0.008701,0.024020,0.023595,0.004469,0.023954,0.033440
2,EN-English,0.033113,0.000948,0.002688,0.024747,0.003913,0.018039
3,ES-Spanish,0.006446,0.003276,0.004322,0.000441,0.003358,0.028456
4,FA-Farsi,0.002247,0.000531,0.019951,0.012296,0.004739,0.014788


In [9]:
langs = entity_counts['lang'].unique()


val_indices = []

for lang in langs:  # работаем с каждым языком по отдельности
    
    # частоты у сущностей, добавляемых первыми, искажаются сильнее
    # поэтому мы начинаем с наиболее "искаженных" сущностей для данного языка
    entity_names = langmeans_diffs.loc[langmeans_diffs['lang'] == lang, entity_names].T.iloc[:,0].sort_values(ascending=False).index.tolist()
    
    
    for entity in entity_names:  # и для каждой сущности
        total = entity_counts.loc[entity_counts['lang'] == lang, entity].sum()
        quota = int(total * 0.15)  # сколько всего текстов нужно отложить в валидацию
        already_added = entity_counts.loc[(entity_counts['lang'] == lang) & entity_counts.index.isin(val_indices), entity].sum()  # сколько текстов из уже добавленных в валидацию содержат эту сущность
        to_add = quota - already_added
        
        # Наблюдается проблема: из-за того, что в одном тексте могут быть несколько сущностей разного вида,
        # мы получаем сильное искажение соотношений для тех сущностей, которые мы добавили в начале. 
        # Чтобы ее сгладить, мы будем понижать вероятность взять в валидацию те тексты, в которых много сущностей разного вида. 
        # Просто линейно 
        
        probabilities = 1 / entity_counts.loc[(entity_counts['lang'] == lang) & (entity_counts[entity] == 1), entity_names].sum(axis=1)
        probabilities /= probabilities.sum()
        
        
        new_val_indices = np.random.choice(entity_counts.loc[(entity_counts['lang'] == lang) & (entity_counts[entity] == 1)].index, to_add, replace=False, p=probabilities)
        val_indices.extend(new_val_indices)

Проверяем корректность разбиения. Аналогично EDA (сейчас опять будет копипаста кода), убеждаемся, что по всем языкам и по всем сущностям мы сохранили их соотношение

In [10]:
entity_counts['type'] = 'train'
entity_counts.loc[val_indices, 'type'] = 'validation'

entity_counts.groupby(['lang', 'type'])[entity_names].mean()

LOC      PROD        CW       GRP      CORP  \
lang       type                                                           
BN-Bangla  train       0.153367  0.208333  0.140990  0.157211  0.169742   
           validation  0.153578  0.208551  0.140925  0.157068  0.169721   
DE-German  train       0.201247  0.170082  0.202805  0.196182  0.184106   
           validation  0.210953  0.163083  0.180933  0.213793  0.174442   
EN-English train       0.207079  0.166783  0.203890  0.207079  0.179463   
           validation  0.213088  0.156646  0.232720  0.188139  0.202454   
ES-Spanish train       0.212638  0.171498  0.216534  0.194016  0.168926   
           validation  0.229116  0.159773  0.194242  0.181671  0.176805   
FA-Farsi   train       0.215679  0.171393  0.199984  0.182892  0.180328   
           validation  0.228489  0.164265  0.193495  0.169205  0.168794   
HI-Hindi   train       0.158867  0.199046  0.147706  0.184960  0.175493   
           validation  0.157279  0.199740  0.146880  0.181976  0.176776   
KO-Korean  train       0.237508  0.162477  0.229388  0.195971  0.193941   
           validation  0.245185  0.170144  0.204655  0.174960  0.195024   
NL-Dutch   train       0.233038  0.169468  0.188220  0.192733  0.165577   
           validation  0.219363  0.165441  0.207516  0.176062  0.180556   
RU-Russian train       0.210204  0.166783  0.187873  0.181670  0.171358   
           validation  0.195589  0.163962  0.204328  0.183521  0.181856   
TR-Turkish train       0.217998  0.174834  0.201870  0.189716  0.164862   
           validation  0.228398  0.163083  0.202840  0.199594  0.167546   
ZH-Chinese train       0.295387  0.275206  0.288063  0.040517  0.226664   
           validation  0.299270  0.267640  0.310219  0.040146  0.210868   

                            PER  
lang       type                  
BN-Bangla  train       0.170357  
           validation  0.170157  
DE-German  train       0.253292  
           validation  0.272617  
EN-English train       0.260132  
           validation  0.250307  
ES-Spanish train       0.240299  
           validation  0.257502  
FA-Farsi   train       0.221195  
           validation  0.260601  
HI-Hindi   train       0.155711  
           validation  0.158579  
KO-Korean  train       0.228061  
           validation  0.250803  
NL-Dutch   train       0.229381  
           validation  0.232843  
RU-Russian train       0.219974  
           validation  0.209738  
TR-Turkish train       0.233580  
           validation  0.210142  
ZH-Chinese train       0.115864  
           validation  0.100973

И так же, как и в EDA, посмотрим на разницу средних

In [11]:
val_langmeans = entity_counts.groupby(['lang', 'type'])[entity_names].mean().reset_index(level=1)
val_langmeans_diffs = (val_langmeans.loc[val_langmeans['type'] == 'train', entity_names] - val_langmeans.loc[val_langmeans['type'] == 'validation', entity_names]).abs()
val_langmeans_diffs.max()

LOC     0.016478
PROD    0.011751
CW      0.028830
GRP     0.021011
CORP    0.022991
PER     0.039406
dtype: float64

В целом, разбиение получилось довольно близким, хотя местами есть довольно заметная разница

In [12]:
langmeans_diffs.set_index('lang') - val_langmeans_diffs

,CORP,CW,GRP,LOC,PER,PROD
lang,,,,,,
BN-Bangla,0.010968,0.008954,0.009547,0.026938,0.009474,0.028916
DE-German,-0.000963,-0.017403,0.006409,0.023733,0.004270,0.016956
EN-English,0.010122,-0.004084,-0.017992,0.012030,-0.007138,-0.006224
ES-Spanish,-0.001432,-0.021851,-0.009069,0.011978,-0.012881,-0.008367
FA-Farsi,-0.009287,0.005807,-0.013155,0.001977,-0.019455,-0.002389
HI-Hindi,0.008152,0.006756,-0.002494,0.005789,0.005989,0.010155
KO-Korean,0.013035,-0.021874,-0.017212,0.008581,0.001743,0.012357
NL-Dutch,0.005798,-0.004354,-0.007855,-0.003275,0.008972,0.009797
RU-Russian,0.000245,-0.013162,0.001360,-0.013774,-0.009370,0.002089


В принципе, удалось разбить немного получше, чем в train/dev разбиении от оргов. 

# Сохраняем разбиение

Тут есть сложность, связанная с тем, что датасет в json может сгенерироваться немного по-разному. 

Самый верный способ: сохранить новое разбиение в том же CONLL, что и исходные данные. 


In [13]:
data[125074]

{'tokens': ['27',
  'декабря',
  '2014',
  'общественная',
  'организация',
  '«',
  'закарпатська',
  'волонтерська',
  'сотня',
  '»',
  'передала',
  'батальону',
  'внедорожник',
  'mitsubishi',
  'l200',
  '.'],
 'token_labels': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PROD',
  'I-PROD',
  'O'],
 'lang': 'RU-Russian',
 'id': '0e4785d0-9fea-4cea-9a95-354152b8e9e4',
 'domain': 'train',
 'type': 'train'}

In [14]:
import os
from pathlib import Path
from tqdm.notebook import tqdm


def sample_to_conll(sample):
    lines = []
    lines.append(f"# id {sample['id']}	domain={sample['domain']}")  # Решил поле "domain" пока не трогать
    for token, token_label in zip(sample['tokens'], sample['token_labels']):
        lines.append(f"{token} _ _ {token_label}")
    return lines

path_to_save_data = Path('../data/processed_data/')

for i, sample in tqdm(enumerate(data), total=len(data)):
    path_to_folder = path_to_save_data / sample['lang']
    if not path_to_folder.exists():
        os.mkdir(path_to_folder)
    
    sample_type = 'validation' if i in val_indices else 'train'
    
    with open(path_to_folder / f'{sample_type}.conll', 'a') as f:
        lines = sample_to_conll(sample)
        f.write('\n'.join(lines))
        f.write('\n\n')

  0%|          | 0/168300 [00:00<?, ?it/s]